In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np, pandas as pd
import pickle
import os

if not os.path.isdir("./lut_plot"):
    os.mkdir("./lut_plot")

In [ ]:
def get_meas(path):
    measurements = pickle.load(open(path, "rb"))
    df = pd.DataFrame(measurements, columns=["ampl","mag_gen_sq","mag_feedback_sq","phase_diff"])
    df["mag_gen"] = np.sqrt(df["mag_gen_sq"])
    df["mag_feedback"] = np.sqrt(df["mag_feedback_sq"])
    return df

In [ ]:
df = get_meas("./measurements.pkl")
df_lut = get_meas("./measurements_lut.pkl")
df_lut_sq = get_meas("./measurements_lut_sq.pkl")

In [ ]:
def get_slope(ampl, mag_feedback):
    slope, intersect = np.polyfit(x = ampl[0:20], y = mag_feedback[0:20], deg = 1)
    return slope, intersect

#get_slope(df["ampl"], df["mag_feedback"])

In [ ]:
def get_fac(ampl, mag_feedback):
    slope, intersect = get_slope(df["ampl", df["mag_feedback"]])
    return[(x*slope + intersect) / y for (x,y) in zip(df["ampl"], df["mag_feedback"])]
def interp(x): return np.interp(x, df["ampl"], fac)

In [ ]:
x = np.linspace(0.1, 0.5, num = 50)

plt.plot(df["ampl"], df["mag_feedback"], label="measurement")
plt.plot(x, x*slope + intersect, label = "linear model")

plt.legend(loc=0)
plt.title("Original Measurement")

plt.savefig("./lut_plot/original_measurement.png")

In [ ]:
slope, intersect = get_slope(df["ampl"], df["mag_feedback"])

plt.plot(df_lut["ampl"], df_lut["mag_feedback"], label="measurement")
plt.plot(x, x*slope + intersect, label = "linear model")

plt.legend(loc=0)
plt.title("Lut Measurement")

plt.savefig("./lut_plot/lut_measurement.png")

In [ ]:
slope, intersect = get_slope(df["ampl"], df["mag_feedback"])

plt.plot(df_lut_sq["ampl"], df_lut_sq["mag_feedback"], label="measurement")
plt.plot(x, x*slope + intersect, label = "linear model")

plt.legend(loc=0)
plt.title("Lut Squared Measurement")

plt.savefig("./lut_plot/lut_sq_measurement.png")

In [ ]:
slope, intersect = get_slope(df["ampl"], df["mag_feedback"])

plt.plot(df["ampl"], df["mag_feedback"], label="measurement")
plt.plot(df_lut["ampl"], df_lut["mag_feedback"], label="measurement lut")
plt.plot(df_lut_sq["ampl"], df_lut_sq["mag_feedback"], label="measurement lut sq")
plt.plot(x, x*slope + intersect, label = "linear model")

plt.legend(loc=0)
plt.title("All Measurements")

plt.savefig("./lut_plot/all_measurement.png")

In [ ]:
pickle.dump({"ampl":df["ampl"],"fac":[f for f in fac]}, open("lut.pkl", "wb"))
pickle.dump({"ampl":df["ampl"],"fac":[f**2 for f in fac]}, open("lut_sq.pkl", "wb"))